<a href="https://colab.research.google.com/github/CSID-DGU/2023-2-OSSP1-CrashLandingOnYou-6/blob/master/foodtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-k4zq68yl/kobert-tokenizer_11bf4d8596c54db58103c9c309edc549
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-k4zq68yl/kobert-tokenizer_11bf4d8596c54db58103c9c309edc549
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
from transformers import BertForSequenceClassification
from kobert_tokenizer import KoBERTTokenizer
import torch
from sklearn.preprocessing import LabelEncoder

# Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 모델 및 토크나이저 경로
model_path = "/content/drive/My Drive/Model/model"
tokenizer_path = "/content/drive/My Drive/Model/tokenizer"  # 토크나이저가 저장된 폴더 경로

# 모델 로드
model = BertForSequenceClassification.from_pretrained(model_path, local_files_only=True)

# 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained(tokenizer_path, local_files_only=True)

# 나머지 코드...

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델을 올바른 디바이스로 이동
model.to(device)

# 레이블 인코더
label_encoder = LabelEncoder()
labels = ['볶음', '무침', '굽기', '끓이기', '비빔', '찜', '삶기', '회', '튀김', '조림', '부침', '기타']
label_encoder.fit(labels)

# 예측을 수행하는 함수
def predict_new_recipe(new_recipe):
    # 토크나이징
    encoded_recipe = tokenizer.encode_plus(
        new_recipe,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # 모델에 입력하기 위해 토크나이징된 데이터를 CUDA로 이동(사용 가능한 경우)

    input_ids = encoded_recipe['input_ids'].to(device)
    attention_mask = encoded_recipe['attention_mask'].to(device)

    # 예측
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

    # 결과 해석
    predicted_label = label_encoder.inverse_transform(predictions.cpu().numpy())[0]
    return predicted_label

# 새로운 레시피에 대한 예측
new_recipe = "팬에 식용유를 두 바퀴 정도 두르고 대파와 당근을 볶다가 당근이 노릇해지면 어묵과 양파를 넣고 볶습니다어묵이 노릇노릇하게 볶아질 때 양념장을 넣고 양념장이 골고루 베어 들도록 중약불에서 약 3분간 섞어가며 볶아주면 완성	."
print(f"예측된 레시피 분류: {predict_new_recipe(new_recipe)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
